In [ ]:
 import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/AD Identification using SATD'

Mounted at /content/drive
/content/drive/My Drive/AD Identification using SATD


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.colors as mcolors
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier



In [ ]:
import pandas as pd
liu_ = '/content/drive/My Drive/AD Identification using SATD/liu_datset_processed.csv'
liu_ = pd.read_csv(liu_, low_memory=False)

In [ ]:
liu_ = '/content/drive/My Drive/AD Identification using SATD/liu_datset_processed.csv'
liu_ = pd.read_csv(liu_, low_memory=False)

# Handle missing values in 'Comments'
liu_['Comments'].fillna('', inplace=True)

# Standardize 'TDType' column values
values_to_remove = ['MULTITHREAD', 'nan', 'removeType']
replacement_value = 'WITHOUT_CLASSIFICATION'
liu_['TDType'] = liu_['TDType'].fillna('').astype(str).replace(values_to_remove, replacement_value)

# Clean 'Comments' column
liu_['Comments'] = liu_['Comments'].str.replace('content=', '', regex=False).str.replace('"', '', regex=False)

# Remove duplicates based on specific columns (e.g., 'Comments' and 'TDType')
liu_ = liu_.drop_duplicates(subset=['Comments', 'TDType'])


In [ ]:
# Count the number of duplicate rows in the DataFrame
num_duplicates = liu_.duplicated().sum()

print(f"Number of duplicate rows: {num_duplicates}")


Number of duplicate rows: 0


#LR TFIDF

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score


data = liu_

# Split data into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(data['Comments'], data['TDType'], test_size=0.2, random_state=42)
X_train_final, X_val, y_train_final, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

pipeline = Pipeline([
    ('TFIDF', TfidfVectorizer()),
    ('scaler', StandardScaler(with_mean=False)),
    ('clf', LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced'))
])

# Define the hyperparameters grid for tuning LR
param_grid = {
    'clf__C': [0.01, 0.1, 1, 10],  # Regularisation strength
    'clf__penalty': ['l2', 'elasticnet'],  # Regularisation penalty type
    'clf__max_iter': [100, 200, 500]  # Number of iterations for convergence
}


grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                           scoring='accuracy',
                           n_jobs=-1,
                           verbose=2)

# Fit the grid search to the training data
grid_search.fit(X_train_final, y_train_final)
print("Best parameters found:", grid_search.best_params_)

# Evaluate the best model on the validation set
best_model = grid_search.best_estimator_
y_val_pred = best_model.predict(X_val)
conf_matrix_val = confusion_matrix(y_val, y_val_pred)
classification_rep_val = classification_report(y_val, y_val_pred)

print("\nValidation Confusion Matrix:")
print(conf_matrix_val)
print("\nValidation Classification Report:")
print(classification_rep_val)

# Evaluate the best model on the test set
y_test_pred = best_model.predict(X_test)
conf_matrix_test = confusion_matrix(y_test, y_test_pred)
classification_rep_test = classification_report(y_test, y_test_pred)

print("\nTest Confusion Matrix:")
print(conf_matrix_test)
print("\nTest Classification Report:")
print(classification_rep_test)


Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best parameters found: {'clf__C': 10, 'clf__max_iter': 200, 'clf__penalty': 'l2'}

Validation Confusion Matrix:
[[  67    0    0   55    0    4    0   17]
 [   2   38    1   18    0    4    0    4]
 [   3    5   55   19    1    4    6    6]
 [  53   27   39 1396    3   94   15   89]
 [   1    0    0    3   12    1    1    2]
 [   5    4    5   79    5  234    4   12]
 [   1    2    2   11    0    2   82    5]
 [  38   13    7  131    1   21   18 3493]]

Validation Classification Report:
                        precision    recall  f1-score   support

             ALGORITHM       0.39      0.47      0.43       143
         COMPATIBILITY       0.43      0.57      0.49        67
                DEFECT       0.50      0.56      0.53        99
                DESIGN       0.82      0.81      0.81      1716
         DOCUMENTATION       0.55      0.60      0.57        20
        IMPLEMENTATION       0.64      0.67      0.66       3